In [60]:
"""
PER LA VISUALIZZAZIONE DEL GIOCO 
            *
           ***
          *****
         *******
        *********
 

-Task3.1: An agent using fixed rules based on nim-sum (i.e., an expert system)
ES: if (num_object > 20 then something)

-Task3.2: An agent using evolved rules
ES: if (a < num_object < b) => ottimizzare a e b come se fossero parametri di un genoma

-Task3.3: An agent using minmax

-Task3.4: An agent using reinforcement learning

"""

'\nPER LA VISUALIZZAZIONE DEL GIOCO \n            *\n           ***\n          *****\n         *******\n        *********\n \n\n-Task3.1: An agent using fixed rules based on nim-sum (i.e., an expert system)\nES: if (num_object > 20 then something)\n\n-Task3.2: An agent using evolved rules\nES: if (a < num_object < b) => ottimizzare a e b come se fossero parametri di un genoma\n\n-Task3.3: An agent using minmax\n\n-Task3.4: An agent using reinforcement learning\n\n'

In [61]:
import random
from collections import namedtuple
POPULATION_SIZE = 20     
PROBLEM_SIZE = 5           
GENETIC_OPERATOR_RANDOMNESS = 0.3
Individual = namedtuple("Individual", ["genome", "fitness"])
num_rows = 5
k = 5
chosen_row = 0
i = 0

FLAG_FIX_RULES = 0
FLAG_EVOL_RULES = 1

In [62]:
def mutation(g):                                
    point = random.randint(0, PROBLEM_SIZE - 1) 
    num_objects = random.randint(1, 5)     
    g = g[:point] + (1 - g[point],) + g[point + 1 :]
    g = list(g)
    g[PROBLEM_SIZE] = num_objects
    g = tuple(g)
    return g

In [63]:
class Nim:
    def __init__(self, num_rows: int, k: int = None):
        self._rows = [i*2 + 1 for i in range(num_rows)]
        self._k = k
        self._num_rows = num_rows
        self._tot_objects = k*k
        self._row_opponent = 0 #l'avversario gioca in maniera standard: toglie sempre 2 oggetti dalla riga in cui ce ne sono di piu
        self._num_objects_opponent = 2
        self.GIOCO_FINITO = False
    
        
    def nimming(self, row: int, num_objects: int): #toglie num_object da una determinata row
        if num_objects <= self._k and self._k is not None: #check per correttezza della mossa
            #se il numero di oggetti da togliere è maggiore di quelli attualmente presenti in quella riga tolgo solo quelli
            if self._rows[row] < num_objects:
                num_objects = self._rows[row]
            
            print(f"tolgo {num_objects} oggetti dalla riga {row}")
            self._rows[row] -= num_objects
            self._tot_objects -= num_objects 
            
            if sum(self._rows) == 0:
                self.GIOCO_FINITO = True
                print("Gioco finito")
        else:
            print("Non hai inserito una mossa valida")
            
    ###TASK 3.1 FIXED RULES####
    def my_turn_fixed_rules(self):
        object_to_remove = 0
        if (self._tot_objects < 25 and self._tot_objects >= 20):
            object_to_remove = 3
        if (self._tot_objects < 20 and self._tot_objects >= 15):
            object_to_remove = 2
        if (self._tot_objects < 15 and self._tot_objects >= 10):
            object_to_remove = 1
        if (self._tot_objects < 10 and self._tot_objects >= 5):
            object_to_remove = 1
        if (self._tot_objects < 5 and self._tot_objects >= 1):
            object_to_remove = 1

        return object_to_remove   
    
    def compute_fitness(self, genome):
        chosen_row = 0
        num_objects = genome[PROBLEM_SIZE]                            
        for i in range(PROBLEM_SIZE):
            if genome[i]==1:
                chosen_row = i
  
        return self._rows[chosen_row] - num_objects

    def find_champion(self):
        population = list()
        for genome in [tuple(0 for _ in range(PROBLEM_SIZE+1)) for _ in range(POPULATION_SIZE)]:
            genome = mutation(genome)     #the initial genomes of the population are created by setting randomly only one element to 1 
            population.append(Individual(genome, self.compute_fitness(genome))) 
        population = sorted(population, key=lambda i: i[1], reverse=True)
        return population[0]
        
    
    ###TASK 3.1 EVOLUTIONARY RULES####
    def my_turn_evol_rules(self):
        champion = self.find_champion()
        print(champion)
        
        chosen_row = 0
        num_objects = champion[0][PROBLEM_SIZE]
        for i in range(PROBLEM_SIZE):
            if champion[0][i] == 1:
                chosen_row = i
        
        return chosen_row, num_objects
           

In [64]:
nim = Nim(num_rows, k)
print(f"comincia il gioco {nim._rows}")
    
while nim.GIOCO_FINITO == False:  #pari => opponent turn, dispari => my turn
    max = 0
    for j in range(nim._num_rows):
        if nim._rows[j] > max:
            max = nim._rows[j]
            chosen_row = j
            nim._row_opponent = chosen_row
            
    if FLAG_FIX_RULES: 
        if i % 2 == 0:
            print(f"turno {i} gioca l'avversario e rimangono {nim._tot_objects} oggetti")
            nim.nimming(chosen_row, nim._num_objects_opponent)
        else:
            print(f"turno {i} gioco io e rimangono {nim._tot_objects} oggetti")
            my_num_objects = nim.my_turn_fixed_rules()
            nim.nimming(chosen_row, my_num_objects)
                
    if FLAG_EVOL_RULES:
        if i % 2 == 0:
            print(f"turno {i} gioca l'avversario e rimangono {nim._tot_objects} oggetti")
            nim.nimming(chosen_row, nim._num_objects_opponent)
        else:
            print(f"turno {i} gioco io e rimangono {nim._tot_objects} oggetti")
            chosen_row, my_num_objects = nim.my_turn_evol_rules()
            nim.nimming(chosen_row, my_num_objects)
        
    print(f"condizione attuale {nim._rows}")
    i = i + 1
    
if i % 2 != 0:
    print(f"Hai tolto l'ultimo oggetto! Vince la partita: io")
else:
    print(f"Hai tolto l'ultimo oggetto! Vince la partita: il mio avversario")

comincia il gioco [1, 3, 5, 7, 9]
turno 0 gioca l'avversario e rimangono 25 oggetti
tolgo 2 oggetti dalla riga 4
condizione attuale [1, 3, 5, 7, 7]
turno 1 gioco io e rimangono 23 oggetti
Individual(genome=(1, 0, 0, 0, 0, 1), fitness=0)
tolgo 1 oggetti dalla riga 0
condizione attuale [0, 3, 5, 7, 7]
turno 2 gioca l'avversario e rimangono 22 oggetti
tolgo 2 oggetti dalla riga 3
condizione attuale [0, 3, 5, 5, 7]
turno 3 gioco io e rimangono 20 oggetti
Individual(genome=(0, 0, 1, 0, 0, 5), fitness=0)
tolgo 5 oggetti dalla riga 2
condizione attuale [0, 3, 0, 5, 7]
turno 4 gioca l'avversario e rimangono 15 oggetti
tolgo 2 oggetti dalla riga 4
condizione attuale [0, 3, 0, 5, 5]
turno 5 gioco io e rimangono 13 oggetti
Individual(genome=(0, 1, 0, 0, 0, 3), fitness=0)
tolgo 3 oggetti dalla riga 1
condizione attuale [0, 0, 0, 5, 5]
turno 6 gioca l'avversario e rimangono 10 oggetti
tolgo 2 oggetti dalla riga 3
condizione attuale [0, 0, 0, 3, 5]
turno 7 gioco io e rimangono 8 oggetti
Individual(g